In [ ]:
# 1. Import Libraries
from google.colab import files
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Input


In [ ]:
url = 'https://drive.google.com/uc?id=1g9IYNBLkNHDNvnLPpKPXDs1hMUGsHkcJ'
df = pd.read_csv(url)

df.head()

In [ ]:
# Preprocess dataset
df = df.dropna()  # remove missing values
df = df[['Latitude', 'Longitude', 'Depth', 'Mag']]  # select useful columns

In [ ]:
X = df[['Latitude', 'Longitude', 'Depth']].values
y = df['Mag'].values

Base Model

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


# User Settings
user_hidden_layers = [16, 8]  # Default (modifiable)
user_batch_size = 16
user_learning_rate = 0.01

feature_indices = {0: 'Latitude', 1: 'Longitude', 2: 'Depth'}
all_features = [0, 1, 2]

# Build Model Function
def build_model(input_dim, hidden_layers, learning_rate):
    model = Sequential()
    model.add(Input(shape=(input_dim,)))  # Add separate Input layer
    model.add(Dense(hidden_layers[0], activation='relu'))  # First hidden layer
    for layer_size in hidden_layers[1:]:
        model.add(Dense(layer_size, activation='relu'))  # Additional hidden layers
    model.add(Dense(1))  # Output layer for regression

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    return model

model = build_model(len(all_features), user_hidden_layers, user_learning_rate)

# Train model
early_stop = EarlyStopping(monitor='val_loss', patience=5)
history = model.fit(X_train, y_train, epochs=500, batch_size=user_batch_size,
                    validation_split=0.2, callbacks=[early_stop], verbose=0)

# Evaluate model
loss, mae = model.evaluate(X_test, y_test)
print(f"Test Mean Absolute Error: {mae:.3f}")

#  Predict on test data
y_pred = model.predict(X_test)

#  Plot true vs predicted
plt.scatter(y_test, y_pred, alpha=0.5)
plt.xlabel("Actual Magnitude")
plt.ylabel("Predicted Magnitude")
plt.title("ANN Earthquake Magnitude Prediction")
plt.grid(True)
plt.show()


In [ ]:
# Feature Selection (Hill Climbing)
print("Starting Feature Selection using Hill Climbing...")

def evaluate_features(features, X_train_full, X_test_full, y_train, y_test):
    X_train_sub = X_train_full[:, features]
    X_test_sub = X_test_full[:, features]

    model = build_model(len(features), user_hidden_layers, user_learning_rate)
    early_stop = EarlyStopping(monitor='val_loss', patience=20, verbose=0)
    history = model.fit(X_train_sub, y_train, validation_split=0.2,
                        epochs=200, batch_size=user_batch_size, verbose=0,
                        callbacks=[early_stop])
    loss, mae = model.evaluate(X_test_sub, y_test, verbose=0)
    return loss, mae

# Start random feature subset
random.seed(42)
np.random.seed(42)

current_features = random.sample(all_features, 2)
print(f"Initial random feature subset: {[feature_indices[i] for i in current_features]}")

best_loss, _ = evaluate_features(current_features, X_train, X_test, y_train, y_test)

improved = True
iteration = 1
while improved:
    improved = False
    print(f"\nFeature Selection Iteration {iteration}")
    for f in all_features:
        if f in current_features:
            candidate = [feat for feat in current_features if feat != f]
        else:
            candidate = current_features + [f]

        if len(candidate) == 0:
            continue

        candidate_loss, _ = evaluate_features(candidate, X_train, X_test, y_train, y_test)

        print(f"Testing candidate features: {[feature_indices[i] for i in candidate]} - Loss: {candidate_loss:.4f}")

        if candidate_loss < best_loss:
            print("Better feature subset found.")
            best_loss = candidate_loss
            current_features = candidate
            improved = True
            break
    iteration += 1

best_features = current_features
print("\nBest feature subset selected:", [feature_indices[i] for i in best_features])

In [ ]:
#  Hyperparameter Tuning (Hill Climbing)
print("\nStarting Hyperparameter Tuning using Hill Climbing...")

neurons_options = [8, 16, 32, 64]
batch_size_options = [16, 32, 64]
learning_rate_options = [0.001, 0.0005, 0.0001]

def evaluate_hyperparameters(hidden_layers, batch_size, learning_rate, X_train_sub, X_test_sub, y_train, y_test):
    model = build_model(X_train_sub.shape[1], hidden_layers, learning_rate)
    early_stop = EarlyStopping(monitor='val_loss', patience=20, verbose=0)
    history = model.fit(X_train_sub, y_train, validation_split=0.2,
                        epochs=200, batch_size=batch_size, verbose=0,
                        callbacks=[early_stop])
    loss, mae = model.evaluate(X_test_sub, y_test, verbose=0)
    return loss, mae

X_train_sub = X_train[:, best_features]
X_test_sub = X_test[:, best_features]

# Start with current user settings
current_hidden_layers = user_hidden_layers.copy()
current_batch_size = user_batch_size
current_learning_rate = user_learning_rate

print(f"Initial hyperparameters: Layers={current_hidden_layers}, Batch Size={current_batch_size}, Learning Rate={current_learning_rate}")

best_loss, _ = evaluate_hyperparameters(current_hidden_layers, current_batch_size, current_learning_rate, X_train_sub, X_test_sub, y_train, y_test)

improved = True
iteration = 1
while improved:
    improved = False
    print(f"\nHyperparameter Tuning Iteration {iteration}")
    candidates = []

    for i in range(len(current_hidden_layers)):
        for neurons in neurons_options:
            if neurons != current_hidden_layers[i]:
                new_layers = current_hidden_layers.copy()
                new_layers[i] = neurons
                candidates.append((new_layers, current_batch_size, current_learning_rate))

    for batch in batch_size_options:
        if batch != current_batch_size:
            candidates.append((current_hidden_layers, batch, current_learning_rate))

    for lr in learning_rate_options:
        if lr != current_learning_rate:
            candidates.append((current_hidden_layers, current_batch_size, lr))

    for hidden_layers_candidate, batch_candidate, lr_candidate in candidates:
        candidate_loss, _ = evaluate_hyperparameters(hidden_layers_candidate, batch_candidate, lr_candidate, X_train_sub, X_test_sub, y_train, y_test)

        print(f"Testing: Layers={hidden_layers_candidate}, Batch={batch_candidate}, LR={lr_candidate} - Loss: {candidate_loss:.4f}")

        if candidate_loss < best_loss:
            print("Better hyperparameter configuration found.")
            best_loss = candidate_loss
            current_hidden_layers = hidden_layers_candidate
            current_batch_size = batch_candidate
            current_learning_rate = lr_candidate
            improved = True
            break
    iteration += 1

print("\nBest Hyperparameters Found:")
print("Hidden layers:", current_hidden_layers)
print("Batch size:", current_batch_size)
print("Learning rate:", current_learning_rate)

In [ ]:
# Final Model Training
print("\nTraining Final Model with Best Settings...")

final_model = build_model(X_train_sub.shape[1], current_hidden_layers, current_learning_rate)
early_stop = EarlyStopping(monitor='val_loss', patience=20, verbose=0)

final_history = final_model.fit(X_train_sub, y_train, validation_split=0.2,
                                epochs=500, batch_size=current_batch_size,
                                verbose=0, callbacks=[early_stop])

final_loss, final_mae = final_model.evaluate(X_test_sub, y_test, verbose=0)
print(f"\nFinal Test Mean Absolute Error (MAE): {final_mae:.3f}")

#  Plot True vs Predicted
print("\nPlotting True vs Predicted Earthquake Magnitudes...")

final_y_pred = final_model.predict(X_test_sub)

plt.figure(figsize=(8,6))
plt.scatter(y_test, final_y_pred, alpha=0.5)
plt.xlabel("Actual Magnitude")
plt.ylabel("Predicted Magnitude")
plt.title("Final Optimized ANN: Earthquake Magnitude Prediction")
plt.grid(True)
plt.show()

# Plot Loss Curves
print("\nPlotting Training and Validation Loss Curves...")

plt.figure(figsize=(8,6))
plt.plot(final_history.history['loss'], label='Training Loss')
plt.plot(final_history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.title('Training and Validation Loss over Epochs')
plt.legend()
plt.grid(True)
plt.show()